<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/HealthInsight_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HealthInsight AI Overview
HealthInsight AI is a system designed to help users get personalized health recommendations based on their health data. The system integrates various agents, each with specific expertise, to process user data, analyze it, and generate detailed insights and recommendations.

Components and Features
User Data Collection Agent: Collects user health data through a user-friendly interface.
Data Preprocessing Agent: Cleans and preprocesses the collected data.
Health Analysis Agent: Analyzes the health data to identify patterns and potential health risks.
Recommendation Agent: Provides personalized health recommendations based on the analysis.
Summary Agent: Generates a concise report summarizing the analysis and recommendations.


## Step-by-Step Implementation
Step 1: Install Required Libraries

In [7]:
!pip install -qU langchain-core==0.2.9
!pip install -qU langchain-community==0.2.5
!pip install -qU 'crewai[tools]'==0.32.0
!pip install -qU langchain-groq==0.1.5
!pip install pandas
!pip install langchain_tools


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.9 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.9 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.32.0 requires langchain<0.2.0,>=0.1.10, but you have langchain 0.2.8 which is incompatible.
crewai-tools 0.3.0 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.2.8 which is incompatible.
embedchain 0.1.109 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.2.8 which is incompatible.


## Step 2: Import Required Dependencies

In [8]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Setup API key for GROQ
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


## Step 3: Setup the LLM
Configure the LLM using ChatGroq:

In [9]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str

def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()

class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> Any:
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",  # or another model name if applicable
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)


## Step 4: Create Tools
Define tools for interacting with the SQL database:

In [11]:
# Establish a database connection
db = SQLDatabase.from_uri("sqlite:///health_data.db")

# Tool to list all tables in the database
@tool("list_tables")
def list_tables() -> str:
    """
    Lists all tables in the connected SQL database.
    """
    return ListSQLDatabaseTool(db=db).invoke("")

# Tool to return the schema and sample rows for given tables
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Returns the schema and sample rows for the given tables.

    Args:
    tables (str): The names of the tables to get schema and sample rows for.
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

# Tool to execute a given SQL query
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """
    Executes a given SQL query on the connected SQL database.

    Args:
    sql_query (str): The SQL query to execute.
    """
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

# Tool to check the SQL query before executing it
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Checks the given SQL query for correctness before executing it.

    Args:
    sql_query (str): The SQL query to check.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})


## Step 5: Create Agents
Define agents with specific roles and tools:

In [12]:
# User Data Collection Agent
data_collector = Agent(
    role="User Data Collector",
    goal="Collect user health data through a user-friendly interface",
    backstory=dedent("""
        You are responsible for collecting user health data through a simple and intuitive interface.
        Ensure data is collected accurately and efficiently.
    """),
    llm=llm,
    tools=[],  # No specific tools needed for data collection
    allow_delegation=False,
)

# Data Preprocessing Agent
data_preprocessor = Agent(
    role="Data Preprocessor",
    goal="Clean and preprocess the collected user health data",
    backstory=dedent("""
        You are responsible for cleaning and preprocessing the collected health data.
        Ensure data is in the correct format and remove any inconsistencies.
    """),
    llm=llm,
    tools=[],  # No specific tools needed for data preprocessing
    allow_delegation=False,
)

# Health Analysis Agent
health_analyst = Agent(
    role="Health Analyst",
    goal="Analyze the health data to identify patterns and potential health risks",
    backstory=dedent("""
        You are responsible for analyzing the user health data to identify any patterns or potential health risks.
        Use advanced analytical techniques to provide insights.
    """),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

# Recommendation Agent
recommendation_agent = Agent(
    role="Recommendation Agent",
    goal="Provide personalized health recommendations based on the analysis",
    backstory=dedent("""
        You are responsible for providing personalized health recommendations based on the analysis of the health data.
        Ensure recommendations are tailored to the user's specific needs.
    """),
    llm=llm,
    tools=[],  # No specific tools needed for providing recommendations
    allow_delegation=False,
)

# Summary Agent
summary_agent = Agent(
    role="Summary Agent",
    goal="Generate a concise report summarizing the analysis and recommendations",
    backstory=dedent("""
        You are responsible for generating a concise report that summarizes the analysis and recommendations.
        Ensure the report is clear and easy to understand.
    """),
    llm=llm,
    tools=[],  # No specific tools needed for summarizing
    allow_delegation=False,
)


## Step 6: Create Tasks
Define tasks for collecting data, preprocessing, analyzing, recommending, and summarizing:

In [13]:
# Task to collect user health data
collect_data = Task(
    description="Collect user health data through a user-friendly interface.",
    expected_output="Collected user health data",
    agent=data_collector,
)

# Task to preprocess the collected data
preprocess_data = Task(
    description="Clean and preprocess the collected user health data.",
    expected_output="Preprocessed user health data",
    agent=data_preprocessor,
    context=[collect_data],
)

# Task to analyze the health data
analyze_data = Task(
    description="Analyze the health data to identify patterns and potential health risks.",
    expected_output="Health data analysis",
    agent=health_analyst,
    context=[preprocess_data],
)

# Task to provide personalized health recommendations
provide_recommendations = Task(
    description="Provide personalized health recommendations based on the analysis.",
    expected_output="Personalized health recommendations",
    agent=recommendation_agent,
    context=[analyze_data],
)

# Task to generate a summary report
generate_summary = Task(
    description=dedent("""
        Generate a concise report summarizing the analysis and recommendations.
        The report must be clear and easy to understand.
    """),
    expected_output="Summary report",
    agent=summary_agent,
    context=[provide_recommendations],
)


## Step 7: Setup the Crew
Create a Crew instance to manage the agents and tasks:

In [14]:
crew = Crew(
    agents=[data_collector, data_preprocessor, health_analyst, recommendation_agent, summary_agent],
    tasks=[collect_data, preprocess_data, analyze_data, provide_recommendations, generate_summary],
    process=Process.sequential,
    verbose=2,
    memory=False,
    output_log_file="health_insight.log",
)


## Step 8: Kickoff the Crew for a Health Query
Provide the health query and start the process:

In [15]:
inputs = {
    "query": "Provide a detailed health insight and recommendations based on my recent health data."
}

result = crew.kickoff(inputs=inputs)

# Display the result
print(result)



 [2024-07-16 15:54:06][DEBUG]: == Working Agent: User Data Collector
 [2024-07-16 15:54:06][INFO]: == Starting Task: Collect user health data through a user-friendly interface.
 [2024-07-16 15:54:11][DEBUG]: == [User Data Collector] Task output: **User Health Data Collection Interface**

**Welcome!**

Please enter your health data to help us better understand your health and provide personalized recommendations.

**Section 1: Demographics**

* **Age:**  _____________________________________
* **Gender:**  _____________________________________
* **Height (in cm):**  _____________________________________
* **Weight (in kg):**  _____________________________________

**Section 2: Medical History**

* **Do you have any pre-existing medical conditions?**  Yes  No
* **If yes, please list:**  _____________________________________
* **Have you had any surgeries?**  Yes  No
* **If yes, please list:**  _____________________________________

**Section 3: Lifestyle**

* **Do you smoke?**  Yes  No
*